In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice_name="fabric_threat_mitigation_" + fablib.get_bastion_username()
slice=fablib.get_slice(slice_name)

In [ ]:
client1 = slice.get_node("client1")
client2 = slice.get_node("client2")
clients = [client1, client2]
for client in clients:
    client.execute("git clone https://github.com/teaching-on-testbeds/AStream > /dev/null 2>&1")
    client.execute("sudo apt update > /dev/null 2>&1")
    client.execute("sudo apt -y install python3 ffmpeg > /dev/null 2>&1")
    client.execute("touch proof.txt")

In [ ]:
attacker1 = slice.get_node("attacker1")
attacker2 = slice.get_node("attacker2")
attacker3 = slice.get_node("attacker3")
attackers = [attacker1, attacker2, attacker3]

for attacker in attackers:
    attacker.execute("sudo apt update > /dev/null 2>&1")
    attacker.execute("sudo apt -y install hping3 > /dev/null 2>&1")
    attacker.execute("touch proof.txt")

# Experiment

In this section, we run the video streaming from clients twice, first time with no ongoing attacks on the network, and the second time with a ddos attack on the server. For the first part, only run the client side code, and for the second part, run Client Side and Attack Sides together.
Try to connect to all the client and attack machines using ssh in parallel using something like tmux, to make it easier to copy paste commands.

## Client Side

In the below section, I have ran the normal mode from client 1 and attack mode from client 2. We could do it on the same too, but getting the result files altogether was easier this way.
> Note: Automate and make this better in future

To start using the video streaming service on any of the clients, log on that client, and run the following command.
> Note: Adjust the timing by changing sleep 60, to something else, if you want to run this for 2 minutes, change it to sleep 120.
```bash
python3 ~/AStream/dist/client/dash_client.py -m http://server/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d & PID=$!; sleep 60; kill $PID
```

# Attack Side

Log in to all the attacker machines, and use 
```bash
sudo su
```
to login as root.
Then execute 
```bash
hping3 -S --flood -p 80 server & PID=$!; sleep 60; kill $PID
```
to lauch the DDoS Attack for 60 seconds. 



# Analysis

## Normal Mode

In [ ]:
client1.execute("cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
slice.get_node("client1").download_file("/home/fabric/work/fabric-threat-mitigation/csv/DASH_BUFFER_LOG_NORMAL.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

## Attack Mode

In [ ]:
client2.execute("cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
slice.get_node("client2").download_file("/home/fabric/work/fabric-threat-mitigation/csv/DASH_BUFFER_LOG_ATTACK.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

## Draw the graph

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Define colors for states
c = {'INITIAL_BUFFERING': 'violet', 'PLAY': 'lightcyan', 'BUFFERING': 'lightpink'}

# Load the first dataset
dash1 = pd.read_csv("/home/fabric/work/fabric-threat-mitigation/csv/DASH_BUFFER_LOG_NORMAL.csv")
dash1 = dash1.loc[dash1.CurrentPlaybackState.isin(c.keys())]
states1 = pd.DataFrame({
    'startState': dash1.CurrentPlaybackState[0:-2].values,
    'startTime': dash1.EpochTime[0:-2].values,
    'endState': dash1.CurrentPlaybackState[1:-1].values,
    'endTime': dash1.EpochTime[1:-1].values
})

# Load the second dataset
dash2 = pd.read_csv("/home/fabric/work/fabric-threat-mitigation/csv/DASH_BUFFER_LOG_ATTACK.csv")
dash2 = dash2.loc[dash2.CurrentPlaybackState.isin(c.keys())]
states2 = pd.DataFrame({
    'startState': dash2.CurrentPlaybackState[0:-2].values,
    'startTime': dash2.EpochTime[0:-2].values,
    'endState': dash2.CurrentPlaybackState[1:-1].values,
    'endTime': dash2.EpochTime[1:-1].values
})

# Create a new figure with a 2x2 layout
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# First subplot: Bitrate for first file
for index, s in states1.iterrows():
    axs[0, 0].axvspan(s['startTime'], s['endTime'], color=c[s['startState']], alpha=1)

axs[0, 0].plot(dash1[dash1.Action != "Writing"].EpochTime, dash1[dash1.Action != "Writing"].Bitrate, 'kx:')
axs[0, 0].set_title("Video rate (bps) - Normal Traffic")
axs[0, 0].set_xlabel("Time (s)")

# Second subplot: Buffer for first file
for index, s in states1.iterrows():
    axs[0, 1].axvspan(s['startTime'], s['endTime'], color=c[s['startState']], alpha=1)

axs[0, 1].plot(dash1[dash1.Action != "Writing"].EpochTime, dash1[dash1.Action != "Writing"].CurrentBufferSize, 'kx:')
axs[0, 1].set_title("Buffer (segments) - Normal Traffic")
axs[0, 1].set_xlabel("Time (s)")

# Add common caption for the first two subplots
fig.text(0.5, 0.5, "Normal Traffic", ha='center', fontsize=14, va='bottom')

# Third subplot: Bitrate for second file
for index, s in states2.iterrows():
    axs[1, 0].axvspan(s['startTime'], s['endTime'], color=c[s['startState']], alpha=1)

axs[1, 0].plot(dash2[dash2.Action != "Writing"].EpochTime, dash2[dash2.Action != "Writing"].Bitrate, 'kx:')
axs[1, 0].set_title("Video rate (bps) - DDoS Attack")
axs[1, 0].set_xlabel("Time (s)")

# Fourth subplot: Buffer for second file
for index, s in states2.iterrows():
    axs[1, 1].axvspan(s['startTime'], s['endTime'], color=c[s['startState']], alpha=1)

axs[1, 1].plot(dash2[dash2.Action != "Writing"].EpochTime, dash2[dash2.Action != "Writing"].CurrentBufferSize, 'kx:')
axs[1, 1].set_title("Buffer (segments) - DDoS Attack")
axs[1, 1].set_xlabel("Time (s)")

# Add common caption for the last two subplots
fig.text(0.5, 0, "DDoS Traffic", ha='center', fontsize=14, va='bottom')
plt.subplots_adjust(hspace=50, wspace=50)
plt.tight_layout(rect=[0, 0, 1, 1])  # Adjust layout to make room for captions

# Save the figure as an image
plt.savefig("plot_output.png", format='png', dpi=300, bbox_inches='tight')  # Adjust the filename and format as needed

plt.show()